In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn import preprocessing 
from catboost import CatBoostClassifier 
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score,mean_squared_error, r2_score
import warnings
warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')
sample = pd.read_csv('SampleSubmission.csv')

In [3]:
train

,Applicant_ID,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,...,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50,default_status
0,Apcnt_1000000,3436.0,0.28505,1.6560,0.0000,0.000,0.0,10689720.0,252072.0,4272776.0,...,0.392854,2.02,0.711632,0.0,0.0,charge,NaN,1.129518,0.044335,no
1,Apcnt_1000004,3456.0,0.67400,0.2342,0.0000,0.000,0.0,898979.0,497531.0,9073814.0,...,0.314281,8.08,0.183584,NaN,0.0,charge,349.805730,1.620483,0.322436,no
2,Apcnt_1000008,3276.0,0.53845,3.1510,0.0000,6.282,NaN,956940.0,NaN,192944.0,...,0.162965,18.18,0.791136,0.0,0.0,charge,NaN,1.513370,0.011640,yes
3,Apcnt_1000012,3372.0,0.17005,0.5050,0.0000,0.000,192166.0,3044703.0,385499.0,3986472.0,...,0.488884,2.02,0.685168,NaN,0.0,charge,89.940100,0.664452,0.082729,no
4,Apcnt_1000016,3370.0,0.77270,1.1010,0.0000,0.000,1556.0,214728.0,214728.0,1284089.0,...,0.275000,12.12,0.438168,0.0,0.0,charge,97.887502,1.427891,0.045630,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55995,Apcnt_999968,3740.0,0.01730,0.0000,0.0000,0.000,770998.0,9637475.0,4047934.0,11641992.0,...,0.000000,9.09,0.730040,NaN,0.0,lending,591.048913,0.000000,0.241598,no
55996,Apcnt_999972,3360.0,2.01145,0.6252,0.0000,0.000,NaN,927765.0,NaN,NaN,...,0.297913,30.30,0.498208,NaN,0.0,lending,NaN,0.000000,NaN,yes
55997,Apcnt_999980,3500.0,0.76640,0.0000,0.0000,0.000,118645.0,3662435.0,3662435.0,3585024.0,...,0.550000,3.03,0.450512,0.0,0.0,lending,1252.474467,2.000000,0.163153,no
55998,Apcnt_999988,3280.0,0.05235,2.0916,2.2212,0.000,NaN,3458599.0,NaN,115533.0,...,0.942854,0.00,0.705048,1.0,0.0,charge,NaN,0.000000,0.002997,no


In [4]:
test.head()

,Applicant_ID,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,...,form_field41,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50
0,Apcnt_1000032,3236.0,0.34875,10.2006,0.0000,0.0,418564.0,418564.0,418564.0,540710.0,...,NaN,0.825000,1.01,0.800000,NaN,0.0,charge,NaN,0.000000,0.011221
1,Apcnt_1000048,3284.0,1.27360,2.9606,9.0198,0.0,0.0,9858816.0,49014.0,1510098.0,...,18.8415,0.507694,4.04,0.623248,1.0,0.0,lending,NaN,0.504974,0.043525
2,Apcnt_1000052,NaN,0.27505,0.0600,0.0000,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,0.00,NaN,NaN,NaN,charge,NaN,0.000000,NaN
3,Apcnt_1000076,3232.0,0.28505,2.8032,0.0000,0.0,0.0,473802.0,473802.0,1724437.0,...,NaN,0.916663,2.02,0.464224,NaN,NaN,charge,90.163742,0.788809,0.104029
4,Apcnt_1000080,3466.0,2.09545,0.8318,2.5182,0.0,19839.0,1150662.0,1150662.0,7860523.0,...,NaN,0.234047,23.23,0.726688,0.0,0.0,lending,1303.587148,1.637733,0.163124


In [5]:
train.shape

(56000, 52)

In [6]:
test.shape

(24000, 51)

In [7]:
train.isnull().sum()/len(train)*100

Applicant_ID       0.000000
form_field1        4.516071
form_field2        6.864286
form_field3        0.633929
form_field4        0.633929
form_field5        0.633929
form_field6       23.857143
form_field7        9.219643
form_field8       23.857143
form_field9       14.300000
form_field10       0.633929
form_field11      56.108929
form_field12      17.669643
form_field13      10.516071
form_field14       0.000000
form_field15      40.133929
form_field16      23.278571
form_field17      19.912500
form_field18      18.575000
form_field19       0.007143
form_field20       0.633929
form_field21      28.310714
form_field22      36.428571
form_field23      50.219643
form_field24      23.744643
form_field25       9.732143
form_field26      13.282143
form_field27      16.605357
form_field28       0.633929
form_field29       0.633929
form_field30      45.551786
form_field31      70.371429
form_field32       9.732143
form_field33       2.242857
form_field34       0.633929
form_field35      41

In [8]:
train.drop(['form_field40', 'form_field31', 'form_field41', 'form_field11', 'form_field45', 'form_field23'],axis=1, inplace = True)
test.drop(['form_field40', 'form_field31', 'form_field41', 'form_field11', 'form_field45', 'form_field23'],axis=1, inplace = True)

In [9]:
train.fillna(-99999, inplace=True)
test.fillna(-99999, inplace=True)

In [10]:
train.isnull().sum()

Applicant_ID      0
form_field1       0
form_field2       0
form_field3       0
form_field4       0
form_field5       0
form_field6       0
form_field7       0
form_field8       0
form_field9       0
form_field10      0
form_field12      0
form_field13      0
form_field14      0
form_field15      0
form_field16      0
form_field17      0
form_field18      0
form_field19      0
form_field20      0
form_field21      0
form_field22      0
form_field24      0
form_field25      0
form_field26      0
form_field27      0
form_field28      0
form_field29      0
form_field30      0
form_field32      0
form_field33      0
form_field34      0
form_field35      0
form_field36      0
form_field37      0
form_field38      0
form_field39      0
form_field42      0
form_field43      0
form_field44      0
form_field46      0
form_field47      0
form_field48      0
form_field49      0
form_field50      0
default_status    0
dtype: int64

In [11]:
test.isnull().sum()

Applicant_ID    0
form_field1     0
form_field2     0
form_field3     0
form_field4     0
form_field5     0
form_field6     0
form_field7     0
form_field8     0
form_field9     0
form_field10    0
form_field12    0
form_field13    0
form_field14    0
form_field15    0
form_field16    0
form_field17    0
form_field18    0
form_field19    0
form_field20    0
form_field21    0
form_field22    0
form_field24    0
form_field25    0
form_field26    0
form_field27    0
form_field28    0
form_field29    0
form_field30    0
form_field32    0
form_field33    0
form_field34    0
form_field35    0
form_field36    0
form_field37    0
form_field38    0
form_field39    0
form_field42    0
form_field43    0
form_field44    0
form_field46    0
form_field47    0
form_field48    0
form_field49    0
form_field50    0
dtype: int64

In [12]:
for i in train.columns:
    print(train[i].dtype)

object
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
int64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
object
float64
float64
float64
object


In [13]:
def replace_form_field(val):
    if val =='charge':
        return 1
    else:
        return 0
train['form_field47']= train['form_field47'].apply(replace_form_field, 1)

In [14]:
def replace_form_field(val):
    if val =='charge':
        return 0
    else:
        return 1
test['form_field47']= test['form_field47'].apply(replace_form_field, 1)

In [15]:
def replace_default_status(val):
    if val =='yes':
        return 1
    else:
        return 0
train['default_status']= train['default_status'].apply(replace_default_status, 1)

In [16]:
train.drop(['Applicant_ID'], inplace=True, axis=1)
test.drop(['Applicant_ID'], inplace=True, axis=1)

In [17]:
train.head()

,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,form_field10,...,form_field39,form_field42,form_field43,form_field44,form_field46,form_field47,form_field48,form_field49,form_field50,default_status
0,3436.0,0.28505,1.6560,0.0,0.000,0.0,10689720.0,252072.0,4272776.0,11333126.0,...,0.0,0.392854,2.02,0.711632,0.0,1,-99999.000000,1.129518,0.044335,0
1,3456.0,0.67400,0.2342,0.0,0.000,0.0,898979.0,497531.0,9073814.0,2533168.0,...,0.0,0.314281,8.08,0.183584,0.0,1,349.805730,1.620483,0.322436,0
2,3276.0,0.53845,3.1510,0.0,6.282,-99999.0,956940.0,-99999.0,192944.0,1079864.0,...,0.0,0.162965,18.18,0.791136,0.0,1,-99999.000000,1.513370,0.011640,1
3,3372.0,0.17005,0.5050,0.0,0.000,192166.0,3044703.0,385499.0,3986472.0,3621979.0,...,1.0,0.488884,2.02,0.685168,0.0,1,89.940100,0.664452,0.082729,0
4,3370.0,0.77270,1.1010,0.0,0.000,1556.0,214728.0,214728.0,1284089.0,361770.0,...,0.0,0.275000,12.12,0.438168,0.0,1,97.887502,1.427891,0.045630,0


In [18]:
train.describe()

,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,form_field10,...,form_field39,form_field42,form_field43,form_field44,form_field46,form_field47,form_field48,form_field49,form_field50,default_status
count,56000.000000,56000.000000,56000.000000,56000.000000,56000.000000,5.600000e+04,5.600000e+04,5.600000e+04,5.600000e+04,5.600000e+04,...,56000.000000,56000.000000,56000.000000,56000.000000,56000.000000,56000.000000,5.600000e+04,56000.000000,5.600000e+04,56000.000000
mean,-1181.922589,-6863.704139,-632.876677,-633.075654,-631.978317,4.516156e+05,6.223043e+06,1.976180e+06,1.126383e+07,1.178006e+07,...,-7519.483643,-2362.116859,-1007.708353,-9611.894653,-28399.647714,0.649518,1.540817e+05,-632.879822,4.622706e+05,0.244911
std,21491.517860,25284.695483,7936.761742,7936.746184,7936.840126,1.288346e+06,1.833529e+07,3.618687e+06,1.889007e+07,2.662676e+07,...,26370.798824,15187.817433,10020.625913,29476.393469,45093.674961,0.477125,1.319361e+06,7936.761406,5.241410e+06,0.430038
min,-99999.000000,-99999.000000,-99999.000000,-99999.000000,-99999.000000,-9.999900e+04,-9.999900e+04,-9.999900e+04,-9.999900e+04,-9.999900e+04,...,-99999.000000,-99999.000000,-99999.000000,-99999.000000,-99999.000000,0.000000,-9.999900e+04,-99999.000000,-9.999900e+04,0.000000
25%,3336.000000,0.047700,0.000000,0.000000,0.000000,0.000000e+00,3.248150e+05,0.000000e+00,4.217732e+05,4.170080e+05,...,0.000000,0.000000,2.020000,0.349148,-99999.000000,0.000000,-9.999900e+04,0.000000,7.502524e-03,0.000000
50%,3474.000000,0.226700,0.060000,0.000000,0.000000,4.006700e+04,2.174704e+06,3.765520e+05,3.712227e+06,3.641234e+06,...,0.000000,0.206250,5.050000,0.548080,0.000000,1.000000,5.141139e+01,0.000000,9.600000e-02,0.000000
75%,3610.000000,0.675700,1.270800,0.000000,0.000000,3.023502e+05,6.335740e+06,2.361230e+06,1.403308e+07,1.315909e+07,...,0.000000,0.611116,10.100000,0.739474,0.000000,1.000000,5.096936e+02,1.158480,3.800000e-01,0.000000
max,3900.000000,18.015050,57.371600,91.672200,407.748600,5.313546e+07,2.158794e+09,1.037397e+08,3.200533e+08,2.191361e+09,...,21.000000,2.200000,91.910000,0.800000,5.000000,1.000000,5.321014e+07,28.000000,2.307571e+08,1.000000


In [19]:
train.corr()

,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,form_field10,...,form_field39,form_field42,form_field43,form_field44,form_field46,form_field47,form_field48,form_field49,form_field50,default_status
form_field1,1.000000,0.093841,0.367267,0.367275,0.367299,0.090081,0.075962,0.126271,0.133001,0.099010,...,0.589912,0.427368,0.375329,0.544412,0.274356,-0.041308,0.041406,0.367280,0.023176,-0.067756
form_field2,0.093841,1.000000,0.078882,0.078887,0.078888,-0.135251,-0.032411,-0.088921,-0.026147,-0.027001,...,0.083544,0.070303,0.074776,0.094041,0.105295,0.096772,0.005324,0.078887,0.029122,0.073161
form_field3,0.367267,0.078882,1.000000,1.000000,0.999999,0.034157,0.027521,0.045765,0.047993,0.035599,...,0.280077,0.489792,0.789060,0.244892,0.126830,-0.010070,0.015364,1.000000,0.008558,0.004208
form_field4,0.367275,0.078887,1.000000,1.000000,0.999999,0.034163,0.027526,0.045775,0.048000,0.035608,...,0.280086,0.489789,0.789059,0.244904,0.126889,-0.010073,0.015367,1.000000,0.008559,0.004214
form_field5,0.367299,0.078888,0.999999,0.999999,1.000000,0.034113,0.027478,0.045704,0.047921,0.035544,...,0.280019,0.489805,0.789063,0.244821,0.126780,-0.010033,0.015343,0.999999,0.008545,0.004273
form_field6,0.090081,-0.135251,0.034157,0.034163,0.034113,1.000000,0.210661,0.529277,0.296922,0.262819,...,0.122089,0.023091,0.010402,0.108465,0.000151,-0.142361,0.032284,0.034158,0.068959,-0.178694
form_field7,0.075962,-0.032411,0.027521,0.027526,0.027478,0.210661,1.000000,0.357610,0.293764,0.900909,...,0.098337,0.046098,0.030579,0.105682,0.053442,-0.057531,0.074998,0.027529,0.063827,-0.120591
form_field8,0.126271,-0.088921,0.045765,0.045775,0.045704,0.529277,0.357610,1.000000,0.614295,0.473102,...,0.163602,0.056590,0.037359,0.167273,0.028896,-0.177610,0.136369,0.045788,0.135401,-0.239748
form_field9,0.133001,-0.026147,0.047993,0.048000,0.047921,0.296922,0.293764,0.614295,1.000000,0.498028,...,0.162150,0.091713,0.061209,0.165491,0.093912,-0.176803,0.160451,0.048036,0.226135,-0.228798
form_field10,0.099010,-0.027001,0.035599,0.035608,0.035544,0.262819,0.900909,0.473102,0.498028,1.000000,...,0.125228,0.067155,0.044726,0.136352,0.095053,-0.098032,0.115041,0.035617,0.103390,-0.169560


In [20]:
features = train.select_dtypes(exclude = object). columns.drop(['default_status'])

In [21]:
for col in features:
    train[col].fillna(-999, inplace = True)
    test[col].fillna(-999, inplace = True)

In [22]:
X = train[features]
y = train['default_status']

In [23]:
def metric(y, pred):
    return roc_auc_score(y, pred, labels=[0,1])

In [31]:
n_skf = 15
kf = StratifiedKFold(n_skf)

seed = 2020

params = {'n_estimators': 4000, 
          'learning_rate': 0.01, 
          'max_depth': 8, 
         'eval_metric': 'AUC',
         'random_seed': seed,
         'early_stopping_rounds': 200,
         'use_best_model': True,
}

In [ ]:
score_list = []
score = 0
test_oofs = []

for i, (tr_idx, vr_idx) in enumerate(kf.split(X, y)):
    
    xtrain, ytrain = X.loc[tr_idx, features], y.loc[tr_idx]
    xval, yval = X.loc[vr_idx, features], y.loc[vr_idx]
    
    model = CatBoostClassifier(**params)
    model.fit(xtrain, ytrain, eval_set=[(xval,yval)], verbose=100)
    
    p= model.predict_proba(xval)[:, 1]
    sc= metric(yval, p)
    score_list.append(sc)
    score += sc/n_skf
    
    pred = model.predict_proba(test[features])[:, 1]
    test_oofs.append(pred)
    
    print('Fold {} : {}'.format(i, sc))
    
print()
print()
print('Avg log : ', score)

0:	test: 0.7994793	best: 0.7994793 (0)	total: 75.5ms	remaining: 5m 1s
100:	test: 0.8184379	best: 0.8184379 (100)	total: 5.63s	remaining: 3m 37s
200:	test: 0.8216253	best: 0.8216470 (199)	total: 11.7s	remaining: 3m 41s
300:	test: 0.8234870	best: 0.8234870 (300)	total: 17.5s	remaining: 3m 35s
400:	test: 0.8252056	best: 0.8252056 (400)	total: 23.8s	remaining: 3m 33s
500:	test: 0.8263222	best: 0.8263295 (498)	total: 29.4s	remaining: 3m 25s
600:	test: 0.8272790	best: 0.8272984 (598)	total: 35.3s	remaining: 3m 19s
700:	test: 0.8282168	best: 0.8282342 (696)	total: 41.2s	remaining: 3m 13s
800:	test: 0.8287495	best: 0.8287541 (799)	total: 46.7s	remaining: 3m 6s
900:	test: 0.8294419	best: 0.8294419 (900)	total: 52.5s	remaining: 3m


In [26]:
f'{n_skf} fold CV, score: {score}'

'15 fold CV, score: 0.8409250168896878'

In [27]:
predictions = np.mean(test_oofs,axis=0)

In [28]:
sample["default_status"]= 1-predictions

In [29]:
sample.to_csv('sub_submission.csv', index=False)

In [30]:
sample.head()

,Applicant_ID,default_status
0,Apcnt_1000032,0.845403
1,Apcnt_1000048,0.552656
2,Apcnt_1000052,0.845200
3,Apcnt_1000076,0.376283
4,Apcnt_1000080,0.674597
